In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fno-data/banknifty_fut.parquet
/kaggle/input/fno-data/nifty_opt_df.bin
/kaggle/input/fno-data/bn_opt_df.bin
/kaggle/input/fno-data/nifty_fut.parquet
/kaggle/input/fno-data/holiday_list_2019_2022.csv


In [2]:
from datetime import datetime, timedelta
import datetime as dt
from tqdm.notebook import tqdm
import joblib
import matplotlib.pyplot as plt
import calendar

class config:
    BANKNIFTY_LOT_SIZE = 15
    TIMEFRAME = "1min"
    ENTRY_EXIT_TIMEFRAME = "5min"
    FROM_DATE = "2021-01-01"
    TO_DATE = "2022-12-21"
    INVESTMENT = 10_000_00 
    COMMISSION = 75
    SELL_MARGIN_PER_STRADDLE = 1_50_000  
    NUM_LOTS = int(INVESTMENT / SELL_MARGIN_PER_STRADDLE)

In [3]:
class config: 
    BANKNIFTY_LOT_SIZE = 15 
    TIMEFRAME = "1min" 
    ENTRY_EXIT_TIMEFRAME = "5min" 
    FROM_DATE = "2021-01-01" 
    TO_DATE = "2022-12-21" 
    INVESTMENT = 10_000_00 
    COMMISSION = 75 
    SELL_MARGIN_PER_STRADDLE = 1_50_000
    NUM_LOTS = int(INVESTMENT / SELL_MARGIN_PER_STRADDLE)

In [4]:
holiday_data = pd.read_csv('/kaggle/input/fno-data/holiday_list_2019_2022.csv')
holiday_data.head()

,idx,holiday_name,holiday_date,holiday_day
0,1,Mahashivratri,"March 04, 2019",Monday
1,2,Holi,"March 21, 2019",Thursday
2,3,Mahavir Jayanti,"April 17, 2019",Wednesday
3,4,Good Friday,"April 19, 2019",Friday
4,5,Maharashtra Day,"May 01, 2019",Wednesday


In [5]:
only_thursday = holiday_data.query('holiday_day == "Thursday"').reset_index(drop=True)
only_thursday['holiday_date'] = pd.to_datetime(only_thursday['holiday_date'])
thursday_holiday_list = only_thursday['holiday_date'].apply(lambda x: str(x.date())).tolist()
thursday_holiday_list

['2019-03-21',
 '2019-08-15',
 '2020-04-02',
 '2021-03-11',
 '2021-05-13',
 '2021-08-19',
 '2021-11-04',
 '2022-04-14']

**STEP 1: Getting Nearest Strike Price**

In [6]:
def getNearestStrikePrice(price, nearestMultiple = 50):
    """
    nearestMultiple = 50 for Nifty
    nearestMultiple = 100 for Banknifty
    """
    inputPrice = int(float(price))
    remainder = int(inputPrice % nearestMultiple)
    if remainder < int(nearestMultiple / 2):
        return inputPrice - remainder
    else:
        return inputPrice + (nearestMultiple - remainder)
    
def get_strike_price(close_price, opt_type, quote):
    # get nearest strike
    if quote == "BANKNIFTY":
        nearest_strike = getNearestStrikePrice(close_price)

    # get ATM strike    
    if opt_type == 'CE':
        if quote == "BANKNIFTY":
            atm_strike = nearest_strike 
        
    elif opt_type == 'PE':
        if quote == 'BANKNIFTY':
            atm_strike = nearest_strike

    return atm_strike


In [7]:


def get_fut_df(quote, timeframe):

    data = pd.read_parquet('/kaggle/input/fno-data/banknifty_fut.parquet')
    data['date'] = pd.to_datetime(data['datetime'])
    data.index = data['date']
    data = data.drop(['datetime'], axis=1)

    start_date = datetime(2021, 1, 1)
    end_date = datetime(2022, 12, 21)
    data = data[(data['date'] >= start_date) & (data['date'] <= end_date)]
    
    data['close'] = data['close'].astype(float)
    data['nearest_strike_price'] = data['close'].apply(getNearestStrikePrice)
    
    return data

get_fut_df(quote="BANKNIFTY", timeframe=config.TIMEFRAME).head()

,stock_code,exchange_code,product_type,expiry_date,right,strike_price,open,high,low,close,volume,open_interest,count,date,nearest_strike_price
date,,,,,,,,,,,,,,,
2021-01-01 09:15:00,CNXBAN,NFO,Futures,28-JAN-21,Others,0,31250,31378,31250,31354.7500,48825,1413425,0,2021-01-01 09:15:00,31350
2021-01-01 09:16:00,CNXBAN,NFO,Futures,28-JAN-21,Others,0,31356.3496,31376,31335.5996,31335.5996,28775,1413425,1,2021-01-01 09:16:00,31350
2021-01-01 09:17:00,CNXBAN,NFO,Futures,28-JAN-21,Others,0,31339,31348.5,31330,31344.6504,21075,1445175,2,2021-01-01 09:17:00,31350
2021-01-01 09:18:00,CNXBAN,NFO,Futures,28-JAN-21,Others,0,31345.25,31370,31345.25,31368.0000,16200,1445175,3,2021-01-01 09:18:00,31350
2021-01-01 09:19:00,CNXBAN,NFO,Futures,28-JAN-21,Others,0,31367.9004,31375.8008,31350,31375.8008,17625,1445175,4,2021-01-01 09:19:00,31400


**STEP 2: Using strike price, calculate expiry date using the timestamp**

In [8]:
def get_current_expiry(d, thursday_holiday_list, weekday=3): # 0 = Monday, 1=Tuesday, 2=Wednesday...
    days_ahead = weekday - d.weekday()
    if days_ahead < 0: # after target day already happened this week
        days_ahead += 7
    ideal_exp = d + dt.timedelta(days_ahead)
    if str(ideal_exp) in thursday_holiday_list:
        new_exp = ideal_exp - dt.timedelta(days=1)
        return new_exp
    return ideal_exp

**STEP 3: Creating Option Symbol**

In [9]:
def get_opt_symbol(strike_price, curr_expiry, opt_type, quote):
    if str(curr_expiry.month)[0] == "0":
        curr_month = str(curr_expiry.month)[1]
    else:
        curr_month = str(curr_expiry.month)
    symbol = quote + str(curr_expiry.year)[-2:] + curr_month + f"{curr_expiry:%d}" + str(int(strike_price)) + opt_type 
    return symbol

In [10]:
option_data_dict = joblib.load("/kaggle/input/fno-data/bn_opt_df.bin")

**STEP 4: Logic for taking entry and exiting**

In [11]:
class PortfolioStatisticsTracker():
    def __init__(self):
        self.num_lots = config.NUM_LOTS
        self.trade_type = []
        self.traded_lots = []
        self.lot_size = []
        self.price_list = []
        self.time_list = []
        self.opt_name_traded = []
        self.commission = config.COMMISSION
        self.pnl = {}
        self.investment = config.INVESTMENT
        self.account_value = config.INVESTMENT
        self.equity = config.INVESTMENT
        self.sell_margin_per_straddle = config.SELL_MARGIN_PER_STRADDLE
        
#     def update_num_lots(self):
#         self.num_lots = int(self.account_value / self.sell_margin_per_straddle)
        
    def go_short(self, num_lots, lot_size, opt_df, opt_name, opt_type, idx_to_trade):
        """
        num_lots = number of lots to trade
        lot_size = 50 for Nifty and 75 for BankNifty
        opt_df = option strikeprice dataframe with OHLCV data
        opt_name = eg: NIFTY22MAY14350CE
        opt_type = CE or PE
        """
        idx = idx_to_trade
        total_qty = num_lots * lot_size 
        self.equity += total_qty * float(opt_df['close'][idx]) # ((total_qty * opt_df['close'][idx]) + config.ADD_MARGIN)
        self.account_value += total_qty * float(opt_df['close'][idx]) # ((total_qty * opt_df['close'][idx]) + config.ADD_MARGIN)
        print(f"Entry short {opt_name}, {num_lots} lots @ {float(opt_df['close'][idx])} on {idx}")
        #self.get_account_val()
#         print(f"Account value : {self.get_account_val()}")
        self.trade_type.append(f"ENTRY SHORT {opt_type}")
        self.traded_lots.append(num_lots)
        self.lot_size.append(lot_size)
        self.price_list.append(float(opt_df['close'][idx]))
        self.time_list.append(idx)
        self.opt_name_traded.append(opt_name)
        PortfolioInvPerStockTracker[opt_name] = total_qty * float(opt_df['close'][idx])

        
    def exit_short(self, num_lots, lot_size, opt_df, opt_name, opt_type, idx_to_trade):
        idx = idx_to_trade
        total_qty = num_lots * lot_size
        self.equity -= total_qty * float(opt_df['close'][idx]) # ((total_qty * opt_df['close'][idx]) - config.ADD_MARGIN - config.COMMISSION)
        self.equity -= config.COMMISSION # subtract brokerage after exit
        self.account_value -= total_qty * float(opt_df['close'][idx]) # ((total_qty * opt_df['close'][idx]) - config.ADD_MARGIN - config.COMMISSION)
        self.account_value -= config.COMMISSION # subtract brokerage after exit
        print(f"Exit short {opt_name}, {num_lots} lots @ {float(opt_df['close'][idx])} on {idx}")
        self.trade_type.append(f"EXIT SHORT {opt_type}")
        self.traded_lots.append(num_lots)
        self.lot_size.append(lot_size)
        self.price_list.append(float(opt_df['close'][idx]))
        self.time_list.append(idx)
        self.opt_name_traded.append(opt_name)
        
        # calculate daily pnl
        # entry short at 500, exit at 490, so entry - exit = pnl for short
        if idx in list(self.pnl.keys()):
            self.pnl[idx] += PortfolioInvPerStockTracker[opt_name] - total_qty*float(opt_df['close'][idx])- config.COMMISSION
        else:
            self.pnl[idx] = PortfolioInvPerStockTracker[opt_name] - total_qty * float(opt_df['close'][idx]) - config.COMMISSION
        
        PortfolioInvPerStockTracker.pop(opt_name)
    
    def get_trade_stats(self):
        earning = self.equity - self.investment
        roi = round(earning/self.investment * 100, 2)
        
        backtest_df = pd.DataFrame({
            "option_symbol": self.opt_name_traded,
            "trade_time": self.time_list,
            "trade_type": self.trade_type,
            "num_lots_traded": self.traded_lots,
            "lot_size": self.lot_size,
            "trade_price": self.price_list
        })
        
        pnl_df = pd.DataFrame(list(self.pnl.keys()), columns=['date'])
        pnl_df['pnl'] = list(self.pnl.values())
        
        print(f"Earning from investing Rs {self.investment} with commission = {self.commission} is Rs {round(earning, 2)} (ROI = {roi}%)")
        return backtest_df, pnl_df
    
    def get_account_val(self):
        return self.account_value

**Resampling timeframe if needed**

In [12]:
def get_resampled_df(df, timeframe):
    ohlc_dict = {                                                                                                             
        'open': 'first',                                                                                                    
        'high': 'max',                                                                                                       
        'low': 'min',                                                                                                        
        'close': 'last',                                                                                                    
        'volume': 'sum',
    }

    resampled_df = df.resample(f'{timeframe}', origin='start_day').apply(ohlc_dict)
    resampled_df = resampled_df.dropna()
    return resampled_df

In [25]:
ENTRY_TIMES = [dt.time(hour=9, minute=20, second=0)]

def backtesting(quote):
    print(f"Backtesting for {quote} - {config.TIMEFRAME}")
    PortfolioStats = PortfolioStatisticsTracker()
    df_1min = get_fut_df(quote=quote, timeframe = config.TIMEFRAME)
    df = get_resampled_df(df_1min, timeframe = config.ENTRY_EXIT_TIMEFRAME)
    df['date'] = df.index
    count = 0
    entry_price = 0 
    lot_size = config.BANKNIFTY_LOT_SIZE
    in_position = False
    no_entry_after = dt.time(hour=15, minute=15, second=0)
    entry_after = dt.time(hour=9, minute=20, second=0)
    
    def check_stop_loss(entry_price, current_price):
        stop_loss_level = entry_price * 1.25
        if float(current_price) >= stop_loss_level:
            return True
        return False
    
    exit_triggered = False
    
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        
        if count == 0:
            count += 1
            continue
        else:
            curr_close = row['close']
            #print(curr_close)
            curr_datetime = row['date']
            curr_high = row['high']
            curr_low = row['low']
            current_expiry = get_current_expiry(curr_datetime.date(), thursday_holiday_list)
            exit_by = dt.datetime.combine(current_expiry, dt.time(hour=15, minute=15))
            intraday_squareoff_by_date = curr_datetime.date()
            intraday_squareoff_by = dt.datetime.combine(intraday_squareoff_by_date, dt.time(hour=15, minute=15))
            
            if not in_position and curr_datetime.time() in ENTRY_TIMES:
                
                try:
                    strike_price = get_strike_price(curr_close, opt_type='CE', quote='BANKNIFTY')
                    opt_name_pe = get_opt_symbol(strike_price, current_expiry, opt_type='PE', quote='BANKNIFTY')
                    opt_name_ce = get_opt_symbol(strike_price, current_expiry, opt_type='CE', quote='BANKNIFTY')
        #           print(f"PE: {opt_name_pe}",f"CE: {opt_name_ce}")

                    def process_option(option_name, option_data_dict):
                        if option_name in option_data_dict:
                            option_dataframe = option_data_dict[option_name]
                            option_dataframe['date'] = pd.to_datetime(option_dataframe['datetime'])
                            option_dataframe.index = option_dataframe['date']
                            option_dataframe = option_dataframe.drop(['datetime'], axis=1)
                            opt_df = get_resampled_df(option_dataframe, timeframe=config.ENTRY_EXIT_TIMEFRAME)
                            opt_df['date'] = opt_df.index
                            return opt_df
                        return None


                    opt_df_pe = process_option(opt_name_pe, option_data_dict)
                    opt_df_ce = process_option(opt_name_ce, option_data_dict)

                    if opt_df_pe is not None and opt_df_ce is not None:
                        
                        print(f"Account value : {PortfolioStats.get_account_val()}")

                        PortfolioStats.go_short(config.NUM_LOTS, lot_size, opt_df_pe, opt_name_pe, opt_type='PE', idx_to_trade=idx)
                        PortfolioStats.go_short(config.NUM_LOTS, lot_size, opt_df_ce, opt_name_ce, opt_type='CE', idx_to_trade=idx)

                        PortfolioTradeTracker[opt_name_pe] = [opt_df_pe, lot_size, 'PE']
                        PortfolioTradeTracker[opt_name_ce] = [opt_df_ce, lot_size, 'CE']
                        in_position = True
                        entry_price_list = PortfolioStats.price_list[-1]
                except Exception as e:
                    PortfolioTradeTracker.clear() 
                    print(e)
                
                count += 1
                continue
            
            if in_position:
                # Exit positions based on stop-loss condition
                keys_to_remove = []
                for opt_name_key, df_lot_list in PortfolioTradeTracker.items():
                    opt_df_exit, lot_size, opt_type =  df_lot_list[0], df_lot_list[1], df_lot_list[2]
                    curr_idx_exit = opt_df_exit.query('date >= @curr_datetime').iloc[0]['date']
                    curr_price_exit = opt_df_exit.query('date >= @curr_datetime').iloc[0]['close']
                    if check_stop_loss(entry_price_list, curr_price_exit):
                        print("Stop Loss Triggered for", opt_name_key)
                        PortfolioStats.exit_short(config.NUM_LOTS, lot_size, opt_df_exit, opt_name_key, opt_type=opt_type, idx_to_trade=curr_idx_exit)
                        keys_to_remove.append(opt_name_key)
                # Remove the keys from the dictionary outside the loop
                for key in keys_to_remove:
                    PortfolioTradeTracker.pop(key)
                
                    
            if curr_datetime > intraday_squareoff_by:
                # Exit all positions at the end of the day
                for opt_name_key, df_lot_list in PortfolioTradeTracker.items():
                    opt_df_exit, lot_size, opt_type =  df_lot_list[0], df_lot_list[1], df_lot_list[2]
                    curr_idx_exit = opt_df_exit.query('date >= @curr_datetime').iloc[0]['date']
                    PortfolioStats.exit_short(config.NUM_LOTS, lot_size, opt_df_exit, opt_name_key, opt_type=opt_type, idx_to_trade=curr_idx_exit)
                in_position = False
                PortfolioTradeTracker.clear() 
            count += 1
        
    backtest_df, pnl_df = PortfolioStats.get_trade_stats()
    return backtest_df, pnl_df

In [26]:
PortfolioTradeTracker =  {}
PortfolioInvPerStockTracker = {}
backtest_df_banknifty, pnl_df_banknifty = backtesting(quote='BANKNIFTY')


Backtesting for BANKNIFTY - 1min


  0%|          | 0/33575 [00:00<?, ?it/s]

Account value : 1000000
Entry short BANKNIFTY2110731500PE, 6 lots @ 327.25 on 2021-01-04 09:20:00
Entry short BANKNIFTY2110731500CE, 6 lots @ 300.0 on 2021-01-04 09:20:00
Stop Loss Triggered for BANKNIFTY2110731500PE
Exit short BANKNIFTY2110731500PE, 6 lots @ 408.15 on 2021-01-04 10:45:00
Exit short BANKNIFTY2110731500CE, 6 lots @ 213.35 on 2021-01-04 15:20:00
Account value : 1000367.5
Entry short BANKNIFTY2110731100PE, 6 lots @ 319.55 on 2021-01-05 09:20:00
Entry short BANKNIFTY2110731100CE, 6 lots @ 236.75 on 2021-01-05 09:20:00
Stop Loss Triggered for BANKNIFTY2110731100CE
Exit short BANKNIFTY2110731100CE, 6 lots @ 299.7 on 2021-01-05 09:30:00
Exit short BANKNIFTY2110731100PE, 6 lots @ 78.9 on 2021-01-05 15:20:00
Account value : 1016210.5
Entry short BANKNIFTY2110732200PE, 6 lots @ 176.85 on 2021-01-07 09:20:00
Entry short BANKNIFTY2110732200CE, 6 lots @ 79.7 on 2021-01-07 09:20:00
Stop Loss Triggered for BANKNIFTY2110732200PE
Exit short BANKNIFTY2110732200PE, 6 lots @ 181.85 on 202

In [27]:
backtest_df_banknifty.describe()

,num_lots_traded,lot_size,trade_price
count,856.0,856.0,856.000000
mean,6.0,15.0,296.250701
std,0.0,0.0,153.184071
min,6.0,15.0,0.050000
25%,6.0,15.0,183.125000
50%,6.0,15.0,289.400000
75%,6.0,15.0,395.375000
max,6.0,15.0,955.000000


In [30]:
pnl_df_banknifty.describe()

,pnl
count,419.000000
mean,629.212411
std,9863.524563
min,-23560.500000
25%,-6165.750000
50%,-1623.000000
75%,6337.500000
max,34705.500000
